In [112]:
import pandas as pd
import json
import requests
from tqdm import *
import time

import random
def random_user_agent():
    with open('user_agent_list.txt', 'r', encoding='utf-8') as f:
        user_agents = f.readlines()
        return random.choice(user_agents).strip()
headers = {
    'Accept':'application/json, text/plain, */*',
    'Accept-Encoding':'gzip, deflate, br, zstd',
    'Accept-Language':'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    # If-None-Match:"06c0f14de3c0229b8ad093f25a17f6572"
    # Origin:https://www.uefa.com
    # Priority:u=1, i
    # Referer:https://www.uefa.com/
    # Sec-Ch-Ua:"Not/A)Brand";v="8", "Chromium";v="126", "Microsoft Edge";v="126"
    # Sec-Ch-Ua-Mobile:?0
    # Sec-Ch-Ua-Platform:"Windows"
    # Sec-Fetch-Dest:empty
    # Sec-Fet
    # Sec-Fetch-Site:same-site
    'User-Agent':random_user_agent()
    # X-Api-Key:ceeee1a5bb209502c6c438abd8f30aef179ce669bb9288f2d1cf2fa276de03f4
}

In [113]:
def get_matches_info():
    url = "https://match.uefa.com/v5/matches?competitionId=3&fromDate=2024-06-15&limit=55&offset=0&order=ASC&phase=ALL&seasonYear=2024&toDate=2024-07-15&utcOffset=8"
    
    res = requests.get(url,headers=headers)

    data = json.loads(res.text)
    
    return [i['id'] for i in data]

match_ids = get_matches_info()
match_ids

['2036161',
 '2036162',
 '2036163',
 '2036164',
 '2036167',
 '2036165',
 '2036166',
 '2036170',
 '2036169',
 '2036168',
 '2036171',
 '2036172',
 '2036176',
 '2036173',
 '2036174',
 '2036177',
 '2036178',
 '2036175',
 '2036182',
 '2036179',
 '2036180',
 '2036184',
 '2036183',
 '2036181',
 '2036185',
 '2036186',
 '2036187',
 '2036188',
 '2036192',
 '2036191',
 '2036189',
 '2036190',
 '2036194',
 '2036193',
 '2036196',
 '2036195',
 '2036198',
 '2036197',
 '2036200',
 '2036199',
 '2036202',
 '2036201',
 '2036203',
 '2036204',
 '2036205',
 '2036206',
 '2036208',
 '2036207',
 '2036209',
 '2036210',
 '2036211']

In [19]:
len(match_ids)

51

In [3]:
# 整理一段代码，只做原始数据爬取
def get_match_data(id):
    url = "https://match.uefa.com/v5/matches?matchId=%s" % id
    res = requests.get(url,headers=headers)
    data = json.loads(res.text)[0]

    return data

match_data = []
for i in tqdm(match_ids[:3]):
    match_data.append(get_match_data(i))
    time.sleep(0.5)

match_data

100%|██████████| 3/3 [00:12<00:00,  4.17s/it]


[{'awayTeam': {'associationId': '200117',
   'associationLogoUrl': 'https://img.uefa.com/imgml/uefacom/elements/logos/ma/SCO.svg',
   'bigLogoUrl': 'https://img.uefa.com/imgml/flags/700x700/SCO.png',
   'confederationType': 'UEFA',
   'countryCode': 'SCO',
   'id': '117',
   'idProvider': 'FAME',
   'internationalName': 'Scotland',
   'isPlaceHolder': False,
   'logoUrl': 'https://img.uefa.com/imgml/flags/70x70/SCO.png',
   'mediumLogoUrl': 'https://img.uefa.com/imgml/flags/240x240/SCO.png',
   'organizationId': '200117',
   'teamCode': 'SCO',
   'teamTypeDetail': 'NATIONAL_MEN_TEAM_A',
   'translations': {'countryName': {'EN': 'Scotland',
     'FR': 'Écosse',
     'DE': 'Schottland',
     'ES': 'Escocia',
     'PT': 'Escócia',
     'IT': 'Scozia',
     'RU': 'Шотландия',
     'ZH': '苏格兰',
     'TR': 'Scotland',
     'HU': 'Scotland',
     'CS': 'Scotland',
     'NL': 'Scotland'},
    'displayName': {'EN': 'Scotland',
     'FR': 'Écosse',
     'DE': 'Schottland',
     'ES': 'Escocia',


In [22]:
# 这段代码后续不这样用，把原始数据爬取和整理为dataframe的两步分开
def get_match_data(id):
    url = "https://match.uefa.com/v5/matches?matchId=%s" % id
    res = requests.get(url,headers=headers)
    data = json.loads(res.text)[0]
    #   'id', 比赛id
    #   'awayTeam', 客场球队信息
    #       'id' 客场球队id
    #       'translation'
    #           'countryname'
    #               'ZH' 客场球队中文名称
    #   'homeTeam', 主场球队信息
    #       'id' 主场球队id
    #       'translation'
    #           'countryname'
    #               'ZH' 主场球队中文名称
    #   'score', 比分
    #       'regular' 常规时间比分
    #           'away' 客场球队得分
    #           'home' 主场球队得分
    #       'total' 全场比赛比分
    #           'away' 客场球队得分
    #           'home' 主场球队得分
    #   'playerOfTheMatch', 本场mvp球员
    #       'player'
    #           'translation'
    #               'name'
    #                   'ZH' 球员中文名
    #               'nationalFieldPosition'
    #                   'ZH' 球员位置（此项目中使用国家队位置）
    #   'playerEvents', 球员事件，红黄牌、进球等

    #   'behindClosedDoors', 非公开?不重要
    #   'competition', 比赛基本信息，比如成人组、男子、比赛类别（杯赛）等
    #   'fullTimeAt', 完赛时间
    #   'group', 组
    #   'kickOffTime', 比赛开始时间
    #   'lineupStatus',
    #   'matchAttendance',
    #   'matchNumber',
    #   'matchday', 比赛日信息
    #   'referees', 裁判信息
    #   'round', 系列赛阶段信息，小组赛、淘汰赛等
    #   'seasonYear', 
    #   'sessionNumber',
    #   'stadium', 球馆信息
    #   'status', 比赛状态
    #   'type', 小组赛？这个字段不确定
    #   'winner' 获胜球队

    need_match_data = [
        data['id'],
        data['awayTeam']['id'],
        data['awayTeam']['translations']['countryName']['ZH'],
        data['homeTeam']['id'],
        data['homeTeam']['translations']['countryName']['ZH'],
        data['score']['regular']['away'],
        data['score']['regular']['home'],
        data['score']['total']['away'],
        data['score']['total']['home'],
        data['playerOfTheMatch']['player']['translations']['name']['ZH'],
        data['playerOfTheMatch']['player']['translations']['nationalFieldPosition']['ZH'],
    ]
    return need_match_data

# match_data = get_match_data(2036161)
match_data = []
for i in tqdm(match_ids):
    match_data.append(get_match_data(i))
    time.sleep(0.5)
    
match_data = pd.DataFrame(match_data,columns=['match_id','away_team_id','away_team_name','home_team_id','home_team_name','away_score','home_score','total_away_score','total_home_score','mvp_name','mvp_position'])
match_data.to_excel('..\\data\\match_data.xlsx',index=False)


In [21]:
match_data

['2036161', '117', '苏格兰', '47', '德国', 1, 5, 1, 5, '贾马尔·穆夏拉', '中场球员']

In [118]:
# 爬取球队数据
def get_match_team_data(id):
    url = "https://matchstats.uefa.com/v1/team-statistics/%s" % id
    res = requests.get(url,headers=headers)
    data = json.loads(res.text)[0]
    return data

match_team_data = []
for i in tqdm(match_ids[:2]):
    match_team_data.append({
        'matchId': i, 
        'matchTeamData': get_match_team_data(i)})
    time.sleep(0.5)

match_team_data

100%|██████████| 2/2 [00:03<00:00,  1.58s/it]


[{'matchId': '2036161',
  'matchTeamData': {'statistics': [{'attributes': {'DistanceMeters': '103682'},
     'name': 'distance_covered',
     'translations': {'name': {'EN': 'Distance covered (km)',
       'FR': 'Distance parcourue (km)',
       'DE': 'Zurückgelegte Distanz (km)',
       'ES': 'Distancia recorrida (km)',
       'PT': 'Distância percorrida (km)',
       'IT': 'Distanza coperta (km)',
       'RU': 'Дистанция (км)',
       'ZH': 'distance_covered',
       'TR': 'Distance covered (km)',
       'HU': 'Distance covered (km)',
       'CS': 'Distance covered (km)',
       'NL': 'Distance covered (km)'}},
     'unit': 'KILOMETER',
     'value': '103.68'},
    {'attributes': {'DistanceMeters': '35961'},
     'name': 'distance_covered_in_possession',
     'translations': {'name': {'EN': 'distance covered in possession',
       'FR': 'distance parcourue avec le ballon',
       'DE': 'distance covered in possession',
       'ES': 'distancia recorrida con la posesión',
       'PT': 

In [117]:
match_team_data[0]

{'statistics': [{'attributes': {'DistanceMeters': '103682'},
   'name': 'distance_covered',
   'translations': {'name': {'EN': 'Distance covered (km)',
     'FR': 'Distance parcourue (km)',
     'DE': 'Zurückgelegte Distanz (km)',
     'ES': 'Distancia recorrida (km)',
     'PT': 'Distância percorrida (km)',
     'IT': 'Distanza coperta (km)',
     'RU': 'Дистанция (км)',
     'ZH': 'distance_covered',
     'TR': 'Distance covered (km)',
     'HU': 'Distance covered (km)',
     'CS': 'Distance covered (km)',
     'NL': 'Distance covered (km)'}},
   'unit': 'KILOMETER',
   'value': '103.68'},
  {'attributes': {'DistanceMeters': '35961'},
   'name': 'distance_covered_in_possession',
   'translations': {'name': {'EN': 'distance covered in possession',
     'FR': 'distance parcourue avec le ballon',
     'DE': 'distance covered in possession',
     'ES': 'distancia recorrida con la posesión',
     'PT': 'distância percorrida com a bola',
     'IT': 'distance covered in possession',
     'R

In [1]:
# 数据整理 （json → dataframe）代码测试
import json
import pandas as pd

with open('..\\data\\match_data.json','r',encoding='utf-8') as f:
    js_match_data = json.load(f)
    # match_data = pd.DataFrame(js_match_data,columns=['match_id','away_team_id','away_team_name','home_team_id','home_team_name','away_score','home_score','total_away_score','total_home_score','mvp_name','mvp_position'])

with open('..\\data\\match_team_data.json','r',encoding='utf-8') as f:
    js_match_team_data = json.load(f)

In [10]:
output = []
for match in js_match_team_data:
    match_id = match['match_id']
    for team in ['match_team1_data', 'match_team2_data']:
        team_data = match[team]
        team_id = team_data['teamId']
        for stat in team_data['statistics']:
            stat_name = stat['name']
            stat_value = stat['value']
            try:
                stat_unit = stat['unit']
            except KeyError:
                stat_unit = ''

            try:    
                stat_trans = stat['translations']['name']['EN']
            except KeyError:
                stat_trans = ''
            
            if 'yellow' in stat_name:
                print(f'{stat_name} {stat_value} {stat_unit}')
            # output.append([match_id, team_id, stat_name, stat_value, stat_unit, stat_trans])
# # ipynb的工作路径是code，所以后面写相对路径要先返回上一级根目录，然后进入data目录
# pd.DataFrame(output, columns=['match_id', 'team_id', 'stat_name', 'stat_value', 'stat_unit', 'stat_trans']).to_excel('..\\data\\match_team_data.xlsx',index=False)
    

yellow_cards_last_15 1 
yellow_cards 1 
yellow_cards_last_15 2 
yellow_cards 2 
yellow_cards_last_15 3 
yellow_cards 3 
yellow_cards_last_15 2 
yellow_cards 2 
yellow_cards_last_15 1 
yellow_cards 1 
yellow_cards_last_15 0 
yellow_cards 0 
yellow_cards_last_15 2 
yellow_cards 2 
yellow_cards_last_15 2 
yellow_cards 2 
yellow_cards_last_15 0 
yellow_cards 0 
yellow_cards_last_15 1 
yellow_cards 1 
yellow_cards_last_15 1 
yellow_cards 1 
yellow_cards_last_15 2 
yellow_cards 2 
yellow_cards_last_15 3 
yellow_cards 3 
yellow_cards_last_15 0 
yellow_cards 0 
yellow_cards_last_15 1 
yellow_cards 1 
yellow_cards_last_15 1 
yellow_cards 1 
yellow_cards_last_15 3 
yellow_cards 3 
yellow_cards_last_15 1 
yellow_cards 1 
yellow_cards_last_15 2 
yellow_cards 2 
yellow_cards_last_15 5 
yellow_cards 5 
yellow_cards_last_15 2 
yellow_cards 2 
yellow_cards_last_15 1 
yellow_cards 1 
yellow_cards_last_15 2 
yellow_cards 2 
yellow_cards_last_15 1 
yellow_cards 1 
yellow_cards_last_15 3 
yellow_cards 3 


In [6]:
js_match_team_data[0]['match_team1_data']['statistics'][0]['name']

'distance_covered'

In [2]:
import pandas as pd
match_team_data_long = pd.read_excel('..\\data\\match_team_data_LONG.xlsx', header=0)

In [8]:
match_team_data_long[['stat_name']].drop_duplicates().reset_index(drop=True).to_excel('..\\data\\match_team_data_LONG_stat_name.xlsx',index=False)

In [6]:
match1 = js_match_data[50]
'''
print(match1['id'],
    match1['competition']['translations']['name']['EN'],
    match1['competition']['type'] ,
    match1['round']['secondaryType'],
    match1['round']['translations']['name']['EN'],
    match1['behindClosedDoors'],
    match1['homeTeam']['translations']['countryName']['EN'],
    match1['homeTeam'][id'],
    match1['awayTeam']['translations']['countryName']['EN'],
    match1['awayTeam'][id'],
    match1['score']['regular']['home'],
    match1['score']['regular']['away'],
    match1['score']['total']['home'],
    match1['score']['total']['away'],
    match1['score']['penalty']['home'], # 这俩要try一下
    match1['score']['penalty']['away'], # 这俩要try一下
    match1['winner']['match']['team']['translations']['countryName']['EN'],
    match1['playerOfTheMatch']['player']['translations']['name']['EN']
    )
'''
match1['winner']['match']['team']['translations']['countryName']['EN']
# match1['homeTeam']['translations']['countryName']['EN']




'Spain'

In [57]:
output_lst = []
for match in js_match_data:
    
    match_id = match['id']
    match_competiton_name = match['competition']['translations']['name']['EN']
    match_competiton_type = match['competition']['type'] 
    # match_round_type = match['round']['secondaryType']
    match_round_name = match['round']['translations']['name']['EN']
    match_isBehindClosedDoors = match['behindClosedDoors']
    match_homeTeam_name = match['homeTeam']['translations']['countryName']['EN']
    match_homeTeam_id = match['homeTeam']['id']
    match_awayTeam_name = match['awayTeam']['translations']['countryName']['EN']
    match_awayTeam_id = match['awayTeam']['id']
    match_score_regular_home = match['score']['regular']['home']
    match_score_regular_away = match['score']['regular']['away']
    match_score_total_home = match['score']['total']['home']
    match_score_total_away = match['score']['total']['away']
    
    try:
        match_score_penalty_home = match['score']['penalty']['home'] # 这俩要try一下
        match_score_penalty_away = match['score']['penalty']['away'] # 这俩要try一下
    except KeyError:
        match_score_penalty_home = ''
        match_score_penalty_away = ''
    
    try:
        match_winner_name = match['winner']['match']['team']['translations']['countryName']['EN']
    except KeyError:
        match_winner_name = match['winner']['match']['reason']
    match_playerOfTheMatch = match['playerOfTheMatch']['player']['translations']['name']['EN']
    
    index_lst = [match_id,
                    match_competiton_name, 
                    match_competiton_type,
                    # match_round_type,
                    match_round_name,
                    match_isBehindClosedDoors,
                    match_homeTeam_name,
                    match_homeTeam_id,
                    match_awayTeam_name,
                    match_awayTeam_id,
                    match_score_regular_home,
                    match_score_regular_away,
                    match_score_total_home,
                    match_score_total_away,
                    match_score_penalty_home,
                    match_score_penalty_away,
                    match_winner_name,
                    match_playerOfTheMatch
                    ]

    output_lst.append(index_lst)
    
df_output = pd.DataFrame(output_lst, 
                        columns=['match_id',
                            'match_competiton_name', 
                            'match_competiton_type',
                            # 'match_round_type',
                            'match_round_name',
                            'match_isBehindClosedDoors',
                            'match_homeTeam_name',
                            'match_homeTeam_id',
                            'match_awayTeam_name',
                            'match_awayTeam_id',
                            'match_score_regular_home',
                            'match_score_regular_away',
                            'match_score_total_home',
                            'match_score_total_away',
                            'match_score_penalty_home',
                            'match_score_penalty_away',
                            'match_winner_name',
                            'match_playerOfTheMatch'
                        ])
df_output.to_excel('match_data_WIDE.xlsx', index=False)

In [ ]:
# match['playerEvents']

# 0 scorers
# match1 = js_match_data[0]
# match1['playerEvents']['scorers'][0]['player']['translations']['name']['EN']
# match1['playerEvents']['scorers'][0]['player']['translations']['fieldPosition']['EN']

# # 0 redCards
# match1 = js_match_data[0]
# match1['playerEvents']['redCards'][0]['player']['translations']['name']['EN']
# match1['playerEvents']['redCards'][0]['player']['translations']['fieldPosition']['EN']

# # 2 penaltiesMissed
# match2 = js_match_data[2]
# match2['playerEvents']['penaltiesMissed'][0]['player']['translations']['name']['EN']
# match2['playerEvents']['penaltiesMissed'][0]['player']['translations']['fieldPosition']['EN']

# # 40 penaltyScorers
# match3 = js_match_data[41]
# match3['playerEvents']['penaltyScorers'][0]['player']['translations']['name']['EN']
# match3['playerEvents']['penaltyScorers'][0]['player']['translations']['fieldPosition']['EN']
output = []
for match in js_match_data[:3]:
    for index in ['scorers', 'redCards', 'penaltyScorers', 'penaltiesMissed']:
        try:
            for event in match['playerEvents'][index]:
                match_id = match['id']
                player_name = event['player']['translations']['name']['EN']
                player_position = event['player']['translations']['fieldPosition']['EN']
                event_time_minute = event['time']['minute']
                event_time_second = event['time']['second']
                event_time = f'{event_time_minute:02d}:{event_time_second:02d}'
                # print(f'[{match_id}] {index} : {player_name} ({player_position}) - {event_time_minute:02d}:{event_time_second:02d}')
                output.append([match_id, index, player_name, player_position, event_time])
        except KeyError:
            pass
output

In [ ]:
# match1 = js_match_data[0]

# match1['referees'][0]['person']['translations']['name']['EN']

output = []
for match in js_match_data:
    match_id = match['id']
    for ref in match['referees']:
        ref_name = ref['person']['translations']['name']['EN']
        try:
            ref_country = ref['person']['translations']['countryName']['EN']
        except KeyError:
            ref_country = 'Unknown'
        # 角色的字符串格式稍微整理一下
        ref_role = '_'.join([i.capitalize() for i in ref['role'].split('_')])
        output.append([match_id, ref_name, ref_country, ref_role])
        # print(f'[{match_id}] Referee: {ref_name} ({ref_country}) - {ref_role}')

In [45]:
ref

{'images': {'SMALL_SQUARE': 'https://img.uefa.com/imgml/referees/75x75/256342.jpg'},
 'person': {'gender': 'MALE',
  'id': '256342',
  'translations': {'firstName': {'EN': 'Hugh',
    'FR': 'Hugh',
    'DE': 'Hugh',
    'ES': 'Hugh',
    'PT': 'Hugh',
    'IT': 'Hugh',
    'RU': 'Хью',
    'ZH': 'Hugh',
    'TR': 'Hugh',
    'HU': 'Hugh',
    'CS': 'Hugh',
    'NL': 'Hugh'},
   'lastName': {'EN': 'Dallas',
    'FR': 'Dallas',
    'DE': 'Dallas',
    'ES': 'Dallas',
    'PT': 'Dallas',
    'IT': 'Dallas',
    'RU': 'Даллас',
    'ZH': 'DALLAS',
    'TR': 'Dallas',
    'HU': 'Dallas',
    'CS': 'Dallas',
    'NL': 'Dallas'},
   'name': {'EN': 'Hugh Dallas',
    'FR': 'Hugh Dallas',
    'DE': 'Hugh Dallas',
    'ES': 'Hugh Dallas',
    'PT': 'Hugh Dallas',
    'IT': 'Hugh Dallas',
    'RU': 'Хью Даллас',
    'ZH': '休·克拉克',
    'TR': 'Hugh Dallas',
    'HU': 'Hugh Dallas',
    'CS': 'Hugh Dallas',
    'NL': 'Hugh Dallas'},
   'shortName': {'EN': 'Dallas',
    'FR': 'Dallas',
    'DE': 'Dal